# How to Use: `calc_mandelbrot_channel`


## Log into OpenBB

In [2]:
%load_ext autoreload
%autoreload 2

from humbldata.core.utils.openbb_helpers import obb_login
from openbb import obb
from humbldata.core.utils.env import Env

# obb_login()
obb.account.login(pat=Env().OBB_PAT, remember_me=True)


## Collect Data

In [3]:
from humbldata.toolbox.technical.mandelbrot_channel.helpers import add_window_index
from humbldata.toolbox.toolbox_helpers import log_returns

symbols = ["AAPL", "PCT", "NVDA", "SMCI", "TSLA", "AMD", "SPY", "GLD", "BTC-USD", "ETH-USD"]
rr_list = ["MSFT", "AAPL", "AMZN", "META", "GOOGL", "NFLX", "TSLA", "NVDA"]
sector_symbols = [
    "XLF",
    "XLU",
    "XLK",
    "XLE",
    "XLI",
    "XLV",
    "XLY",
    "XLP",
    "XLB",
    "XLRE",
    "XTL"
]


data = (
    obb.equity.price.historical(
        "AAPL",
        provider="yfinance",
        start_date="2023-01-01",
        end_date="2024-01-01",
        adjusted=True,
    ).to_polars()
).drop(["dividends", "stock_splits"])

data = log_returns(data, _column_name="close")


In [4]:
from humbldata.toolbox.technical.volatility.realized_volatility_model import calc_realized_volatility

data_rv = calc_realized_volatility(
    data=data,
    window="1m",
    method="std",
    grouped_mean=None,
)


## Calculate Mandelbrot Channel


In [10]:
from humbldata.toolbox.technical.mandelbrot_channel.model import calc_mandelbrot_channel

%time
mandelbrot = calc_mandelbrot_channel(
    data,
    window="1m",
    rv_adjustment=True,
    _rv_method="sq",
    _rv_grouped_mean=False,
    _rs_method="RS",
    _live_price=False
).collect()


CPU times: total: 0 ns
Wall time: 0 ns


In [6]:
mandelbrot_og = mandelbrot